In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
device = next(model.parameters()).device
input_text = prompt
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=2048)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜>You are a fact-checking assistant. For the given Claim and Candidate Evidences, determine the correct Claim Label and list the IDs of those evidences you deem relevant.The label is [SUPPORTS, REFUTES, NOT_ENOUGH_INFO, DISPUTED]
"claim-2152": "Venus doesn't have a runaway greenhouse effect",
  "evidences": [
    {"evidence-1018575": "A runaway greenhouse effect involving carbon dioxide and water vapor has long ago been hypothesized to have occurred on Venus, this idea is still largely accepted."},
  ],
label: "REFUTES"
Now, given the following claim and its candidate evidences, please output in the JSON format: {"label":"", "evidences":""}
"claim-2967": "The contribution of waste heat to the global climate is 0.028 W/m2.",
  "evidences": [
    {"evidence-308923": "Global forcing from waste heat was 0.028 W/m2 in 2005."},
    {"evidence-1185839": "It could prove to be the most inexorable, however, if we are fortunate enough to evade all the rest.” Simple global-scale

In [3]:
import json
def build_prompt_with_example(json_path: str) -> str:
    # few‐shot
    example = {
        "claim-2152": {
            "claim_text": "Venus doesn't have a runaway greenhouse effect",
            "ranked_evidences": [
                {
                    "id": "evidence-1018575",
                    "text": (
                        "A runaway greenhouse effect involving carbon dioxide and water vapor "
                        "has long ago been hypothesized to have occurred on Venus, this idea "
                        "is still largely accepted."
                    )
                }
            ],
            "claim_label": "REFUTES",
            "evidences": ["evidence-1018575"]
        }
    }

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    test_claim_id, test_claim = next(iter(data.items()))
    
    # prompt
    prompt_lines = []
    prompt_lines.append("You are a fact-checking assistant. "
                 "For the given Claim and Candidate Evidences, "
                 "determine the correct Claim Label and list the IDs of those evidences you deem relevant."
                 "The label is [SUPPORTS, REFUTES, NOT_ENOUGH_INFO, DISPUTED]" )
    for cid, info in example.items():
        prompt_lines.append(f"\"{cid}\": \"{info['claim_text']}\",")
        prompt_lines.append("  \"evidences\": [")
        for ev in info['ranked_evidences']:
            prompt_lines.append(f"    {{\"{ev['id']}\": \"{ev['text']}\"}},")
        prompt_lines.append("  ],")
        prompt_lines.append(f"label: \"{info['claim_label']}\"")

    prompt_lines.append(f"Now, given the following claim and its candidate evidences, "
                 "please output in the JSON format: {\"label\":\"\", \"evidences\":\"\"}")
    prompt_lines.append(f"\"{test_claim_id}\": \"{test_claim['claim_text']}\",")
    prompt_lines.append("  \"evidences\": [")
    for ev in test_claim['ranked_evidences']:
        prompt_lines.append(f"    {{\"{ev['id']}\": \"{ev['text']}\"}},")
    prompt_lines.append("  ]")
    prompt_lines.append("label:")
    prompt_lines.append("evidences:")
    return "\n".join(prompt_lines)


if __name__ == "__main__":
    prompt = build_prompt_with_example('./data/test-claims-top6-text.json')
    print(prompt)

You are a fact-checking assistant. For the given Claim and Candidate Evidences, determine the correct Claim Label and list the IDs of those evidences you deem relevant.The label is [SUPPORTS, REFUTES, NOT_ENOUGH_INFO, DISPUTED]
"claim-2152": "Venus doesn't have a runaway greenhouse effect",
  "evidences": [
    {"evidence-1018575": "A runaway greenhouse effect involving carbon dioxide and water vapor has long ago been hypothesized to have occurred on Venus, this idea is still largely accepted."},
  ],
label: "REFUTES"
Now, given the following claim and its candidate evidences, please output in the JSON format: {"label":"", "evidences":""}
"claim-2967": "The contribution of waste heat to the global climate is 0.028 W/m2.",
  "evidences": [
    {"evidence-308923": "Global forcing from waste heat was 0.028 W/m2 in 2005."},
    {"evidence-1185839": "It could prove to be the most inexorable, however, if we are fortunate enough to evade all the rest.” Simple global-scale estimates that recen

In [ ]:
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
print(text)
tokens = tokenizer(text)
num_tokens = len(tokens.input_ids)
print(num_tokens)